In [34]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

import numbers
from sklearn import model_selection


In [45]:
# Aquí, a futuro, habría que añadir los datos del Nasdaq, oro y/o Ethereum para ver los resultados
data_for_use = pd.read_csv('./bitcoin_data_for_use_v2.csv', index_col=0)
data_gold = pd.read_csv('./gold_data_for_use.csv', index_col=0)
data_nasdaq = pd.read_csv('./nasdaq_data_for_use.csv', index_col=0)

data_for_use_w_gold = data_for_use.merge(data_gold, left_index=True, right_index=True)
data_for_use_w_nasdaq = data_for_use.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_w_all = data_for_use_w_gold.merge(data_nasdaq, left_index=True, right_index=True)





In [46]:
#aqui se marca lo que se va a usar de verdad para el resto del programa
data_for_use = data_for_use_w_gold

In [47]:
# Partimos los datos en train y test por fechas, sin que sea aleatorio, que es como funcionaría
# en la realidad el modelo
data_no_test, data_test = train_test_split(data_for_use, shuffle=False, test_size=0.2)

# Partimos los datos de train nuevamente en validation y train
data_train, data_valid = train_test_split(data_no_test, shuffle=False, test_size=0.2)

In [48]:
# Separamos la variable que queremos predecir
X_train=data_train.drop(labels=['subida'], axis=1)
y_train=data_train['subida']

X_valid=data_valid.drop(labels=['subida'], axis=1)
y_valid=data_valid['subida']

X_test=data_test.drop(labels=['subida'], axis=1)
y_test=data_test['subida']

In [49]:
# Miramos cuántos días sube el bitcoin sobre el total en cada uno de los segmentos
# esto nos servirá para hacernos una idea de cómo de buenos son los resultados de los modelos
y_train.sum()/y_train.size

0.5683345780433159

In [50]:
y_valid.sum()/y_valid.size

0.48656716417910445

In [51]:
y_test.sum()/y_test.size

0.5513126491646778

In [52]:
#Tenemos que hacer one-hot encoding sobre la columna "dia"
#Sobre el resto aplicamos el standardscaler

number_columns = data_for_use.select_dtypes('number').columns

t=[('dia', 
    OneHotEncoder(handle_unknown='ignore'),
    ['dia'])
    ,('scaler', StandardScaler(),number_columns)
    ]

ct = ColumnTransformer(transformers=t, remainder='passthrough')

#data_for_use_t=ct.fit_transform(data_for_use)

#X_train=ct.fit_transform(X_train)
#X_valid=ct.transform(X_valid)
#X_test=ct.transform(X_test)

In [53]:
def walk_forward_validation (model, X_train_wfv, y_train_wfv, X_test_wfv, y_test_wfv):
    y_pred_wfv = list()
    for i in range(len(y_test_wfv)):
        X_train_wfv_ct=ct.fit_transform(X_train_wfv)
        model.fit(X_train_wfv_ct, y_train_wfv)
        X_test_wfv_ct=ct.transform(X_test_wfv)
        y_pred_next = model.predict(X_test_wfv_ct[i:i+1])
        y_pred_wfv.append(y_pred_next[0])
        X_train_wfv=X_train_wfv.append(X_test_wfv[i:i+1])
        y_train_wfv=y_train_wfv.append(pd.Series(y_test_wfv[i]))
    
    return metrics.confusion_matrix(y_test_wfv, y_pred_wfv), metrics.f1_score(y_test_wfv,y_pred_wfv), metrics.accuracy_score(y_test_wfv,y_pred_wfv)
        
        

In [31]:
lr = LogisticRegression(penalty='l2', solver='lbfgs')

X_train_c=X_train.copy()
y_train_c=y_train.copy()
X_valid_c=X_valid.copy()
y_valid_c=y_valid.copy()


conf_mat, f1sc, accsc = walk_forward_validation(lr, X_train_c, y_train_c, X_valid_c, y_valid_c)

In [32]:
print(conf_mat, f1sc, accsc)

[[ 69 103]
 [ 80  83]] 0.4756446991404011 0.4537313432835821


In [33]:
for vecinos in range(1,31):
    kn = KNeighborsClassifier(n_neighbors=vecinos)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(kn, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Numero de vecinos aplicado: " + str(vecinos))
    print(conf_mat, f1sc, accsc)

Numero de vecinos aplicado: 1
[[76 96]
 [80 83]] 0.4853801169590643 0.4746268656716418
Numero de vecinos aplicado: 2
[[119  53]
 [125  38]] 0.2992125984251968 0.46865671641791046
Numero de vecinos aplicado: 3
[[ 63 109]
 [ 74  89]] 0.49307479224376727 0.4537313432835821
Numero de vecinos aplicado: 4
[[100  72]
 [107  56]] 0.3848797250859107 0.46567164179104475
Numero de vecinos aplicado: 5
[[ 60 112]
 [ 65  98]] 0.5254691689008043 0.4716417910447761
Numero de vecinos aplicado: 6
[[84 88]
 [97 66]] 0.416403785488959 0.44776119402985076
Numero de vecinos aplicado: 7
[[ 57 115]
 [ 71  92]] 0.4972972972972972 0.44477611940298506
Numero de vecinos aplicado: 8
[[78 94]
 [92 71]] 0.4329268292682926 0.44477611940298506
Numero de vecinos aplicado: 9
[[ 52 120]
 [ 71  92]] 0.49066666666666664 0.4298507462686567
Numero de vecinos aplicado: 10
[[ 68 104]
 [ 93  70]] 0.4154302670623145 0.41194029850746267
Numero de vecinos aplicado: 11
[[ 47 125]
 [ 69  94]] 0.49214659685863876 0.4208955223880597
N

KeyboardInterrupt: 

In [ ]:
for grado in range(1,4):
    sv = SVC(probability=True, kernel='poly', degree=grado)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(sv, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Grado aplicado: " + str(grado))
    print(conf_mat, f1sc, accsc)

In [ ]:
for profund in range(1,31):
    dt = DecisionTreeClassifier(max_depth=profund)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(dt, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Maxima profundidad aplicada: " + str(profund))
    print(conf_mat, f1sc, accsc)

In [ ]:
for estimadores in range(1,21):
    adab = AdaBoostClassifier(n_estimators=estimadores)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(adab, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Estimadores aplicados: " + str(estimadores))
    print(conf_mat, f1sc, accsc)

In [63]:
dt = DecisionTreeClassifier(max_depth=16)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
dt.fit(X_train_c,y_train_c)
y_predict=dt.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[718,  32],
        [ 24, 900]]),
 0.9698275862068965,
 0.966547192353644)

In [64]:
dt = DecisionTreeClassifier(max_depth=20)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
dt.fit(X_train_c,y_train_c)
y_predict=dt.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[748,   2],
        [  0, 924]]),
 0.9989189189189189,
 0.998805256869773)

In [66]:
adab = AdaBoostClassifier(n_estimators=14)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
adab.fit(X_train_c,y_train_c)
y_predict=adab.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[294, 456],
        [185, 739]]),
 0.6974988201982067,
 0.6170848267622461)

In [58]:
kn = KNeighborsClassifier(n_neighbors=12)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[403, 347],
        [295, 629]]),
 0.6621052631578948,
 0.6164874551971327)

In [59]:
kn = KNeighborsClassifier(n_neighbors=20)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[304, 446],
        [229, 695]]),
 0.6731234866828087,
 0.5967741935483871)

In [18]:
print(X_test.iloc[[3]])

             varP0_x   varP1_x   varP2_x   varP3_x   varP4_x   varP5_x  \
2020-08-04 -0.359726  1.743632 -6.003427  3.851523  1.910259  0.096808   

             varP6_x   varP7_x    varP8_x  varP9_x  ...   varPs60   varPm65  \
2020-08-04  1.719487 -0.710133  10.961007  2.35663  ...  0.908823  4.232023   

             varPm85  varPm105  varPm125  varPm145  varPm165  varPm185  \
2020-08-04 -2.442391   6.97137 -0.890699  6.425621 -0.263371 -1.095378   

            varPm205  varPm225  
2020-08-04  0.462994 -0.996479  

[1 rows x 117 columns]


In [54]:
#Este es el código para ver cómo funciona el modelo de verdad

adab = AdaBoostClassifier(n_estimators=14)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)
X_test_c=X_test.copy()
y_test_c=y_test.copy()

conf_mat, f1sc, accsc = walk_forward_validation(adab, X_train_c, y_train_c, X_test_c, y_test_c)

print(conf_mat, f1sc, accsc)

[[ 57 131]
 [ 68 163]] 0.620952380952381 0.5250596658711217
